In [2]:
import pymorphy2
import pandas as pd
from tqdm import tqdm

In [3]:
dataset = pd.read_csv('dataset_2.csv')

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = str(text).split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return ' '.join(res)

Удаляем пунктуацию:

In [6]:
import string

dataset['text'] = dataset['text'].apply(lambda text: str(text).translate(str.maketrans('', '', string.punctuation + '—' + '«' + '»' + '…' + '')))

Лемматизируем:

In [10]:
tqdm.pandas()

In [11]:
dataset['text'] = dataset['text'].progress_apply(lemmatize)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 10385/10385 [03:50<00:00, 45.12it/s]


In [13]:
def update_dict(freq_dict, word):
    if word in freq_dict:
        freq_dict[word] += 1
    else:
        freq_dict[word] = 1


def add_words(freq_dict, words):
    for word in words:
        update_dict(freq_dict, word)


def add_pairs(freq_dict, words):
    for i in range(0, len(words) - 1, 1):
        update_dict(freq_dict, words[i] + ' ' + words[i+1])

Применям к каждой странице функцию add_pairs, которая подсчитывает пары скользящим окном:

In [14]:
collocations = {}

dataset['text'].progress_apply(lambda text: add_pairs(collocations, text.split()))

100%|███████████████████████████████████████████████████████████████████████████████████████████| 10385/10385 [00:13<00:00, 772.91it/s]


0        None
1        None
2        None
3        None
4        None
         ... 
10380    None
10381    None
10382    None
10383    None
10384    None
Name: text, Length: 10385, dtype: object

In [15]:
collocations

{'внимание это': 58,
 'это старый': 63,
 'старый дискуссия': 30,
 'дискуссия который': 37,
 'который некогда': 58,
 'некогда вестись': 30,
 'вестись на': 209,
 'на сайт': 1648,
 'сайт луркоморье': 45,
 'луркоморье пожалуйста': 30,
 'пожалуйста для': 34,
 'для продолжение': 71,
 'продолжение обратиться': 30,
 'обратиться к': 375,
 'к актуальный': 32,
 'актуальный обсуждениекоровать': 1,
 'обсуждениекоровать который': 1,
 'который проводиться': 70,
 'проводиться в': 161,
 'в стиль': 1513,
 'стиль пленарный': 30,
 'пленарный заседание': 35,
 'заседание httpskorovanvvmspace': 1,
 'httpskorovanvvmspace я': 1,
 'я таки': 27,
 'таки запилить': 29,
 'запилить игрушка': 2,
 'игрушка в': 77,
 'в который': 9913,
 'который можно': 2378,
 'можно грабить': 137,
 'грабить корован': 272,
 'корован быть': 8,
 'быть как': 854,
 'как зомби': 15,
 'зомби и': 127,
 'и вообще': 4568,
 'вообще можно': 115,
 'можно делать': 141,
 'делать всё': 380,
 'всё сделать': 113,
 'сделать игра': 85,
 'игра в': 2329,
 '

Подсчитываем частоты отдельных слов:

In [17]:
word_freqs= {}

dataset['text'].progress_apply(lambda text: add_words(word_freqs, text.split()))

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10385/10385 [00:05<00:00, 2026.65it/s]


0        None
1        None
2        None
3        None
4        None
         ... 
10380    None
10381    None
10382    None
10383    None
10384    None
Name: text, Length: 10385, dtype: object

In [19]:
words_total = 0

for word in word_freqs:
    words_total += word_freqs[word]

print(words_total)

20208111


Подсчитываем метрику mutual information:

In [22]:
import math

data = {'Первое слово': [], 'Второе слово': [], 'Частота коллокации': [], 'Частота первого слова': [], 'Частота второго слова': [], 'MI': []}

for collocation in tqdm(collocations):
    if(collocations[collocation] > 1):
        first_word = collocation.split()[0]
        second_word = collocation.split()[1]
        mi = math.log((collocations[collocation] * words_total) / (word_freqs[first_word] * word_freqs[second_word]), 2)
        
        data['Первое слово'].append(first_word)
        data['Второе слово'].append(second_word)
        data['Частота коллокации'].append(collocations[collocation])
        data['Частота первого слова'].append(word_freqs[first_word])
        data['Частота второго слова'].append(word_freqs[second_word])
        data['MI'].append(mi)
        

100%|████████████████████████████████████████████████████████████████████████████████████| 7465374/7465374 [00:09<00:00, 828540.77it/s]


In [23]:
dataset = pd.DataFrame.from_dict(data)

In [24]:
dataset

,Первое слово,Второе слово,Частота коллокации,Частота первого слова,Частота второго слова,MI
0,внимание,это,58,4285,147228,0.893640
1,это,старый,63,147228,7648,0.177152
2,старый,дискуссия,30,7648,773,6.680130
3,дискуссия,который,37,773,119944,3.011558
4,который,некогда,58,119944,993,3.298761
...,...,...,...,...,...,...
1646613,канадский,парламент,2,340,355,8.387365
1646614,ядерный,боезаряд,6,1806,11,12.575380
1646615,сокращение,снв,2,614,5,13.684408
1646616,боезаряд,стратегический,2,11,540,12.732184


In [38]:
dataset[dataset['Частота коллокации'] > 80].sort_values('MI', ascending=False).head(50)

,Первое слово,Второе слово,Частота коллокации,Частота первого слова,Частота второго слова,MI
375693,jojos,bizarre,87,88,131,17.218520
79879,ace,attorney,82,121,91,17.199325
35375,tomb,raider,108,123,122,17.150067
16237,монтить,пайтон,83,114,106,17.082660
345997,saints,row,101,107,139,17.066235
486887,doesnt,afraid,104,129,120,17.050753
1189662,цзян,чэн,82,152,86,16.951791
66000,planescape,torment,82,118,111,16.948924
66005,duke,nukem,142,166,146,16.853314
34547,cyberpunk,2077,88,129,119,16.821818


Видим очень много английских названий и имен. Нам это не очень интересно, поэтому попробуем выкинуть все слова не из русского языка:

In [47]:
import math

data = {'Первое слово': [], 'Второе слово': [], 'Частота коллокации': [], 'Частота первого слова': [], 'Частота второго слова': [], 'MI': []}
alphabet=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')

for collocation in tqdm(collocations):
    if(not alphabet.isdisjoint(collocation)):
        first_word = collocation.split()[0]
        second_word = collocation.split()[1]
        mi = math.log((collocations[collocation] * words_total) / (word_freqs[first_word] * word_freqs[second_word]), 2)
        
        data['Первое слово'].append(first_word)
        data['Второе слово'].append(second_word)
        data['Частота коллокации'].append(collocations[collocation])
        data['Частота первого слова'].append(word_freqs[first_word])
        data['Частота второго слова'].append(word_freqs[second_word])
        data['MI'].append(mi)
        

100%|████████████████████████████████████████████████████████████████████████████████████| 7465374/7465374 [00:30<00:00, 247624.83it/s]


In [48]:
dataset = pd.DataFrame.from_dict(data)

In [67]:
dataset[dataset['Частота коллокации'] > 20].sort_values('MI', ascending=False)[650:700]

,Первое слово,Второе слово,Частота коллокации,Частота первого слова,Частота второго слова,MI
75887,козёл,отпущение,108,922,145,13.994787
416468,алиса,селезнёв,58,817,88,13.992788
257522,словесный,понос,96,293,407,13.989752
343106,ковровый,бомбардировка,30,117,319,13.987544
1601405,фридрих,ницше,25,162,192,13.987475
99219,филипп,киркоров,44,349,157,13.986159
510728,дурацкий,колпак,37,318,145,13.985092
220414,корн,неверно,23,151,190,13.983733
527019,тренажёрный,зал,21,26,1008,13.983029
150458,кинг,гидора,28,546,64,13.983029


In [74]:
dataset[(dataset['Частота коллокации'] > 20) & (dataset['MI'] < 12)].sort_values('MI', ascending=False)[50:100]

,Первое слово,Второе слово,Частота коллокации,Частота первого слова,Частота второго слова,MI
362651,коль,скоро,26,110,1240,11.911387
495281,роман,злотников,28,3975,37,11.909594
150756,макаронный,монстр,52,81,3373,11.909204
32556,страница,неоднозначность,63,3065,108,11.909152
2657211,чугуниевый,бомба,21,61,1812,11.906644
483427,непризнанный,гений,29,123,1244,11.903127
53366,увенчаться,успех,53,90,3109,11.902264
1672051,чудом,спастись,29,383,400,11.901355
272484,млечный,путь,53,56,5001,11.900996
112654,просраться,кирпич,39,118,1748,11.899701


In [103]:
dataset[(dataset['Первое слово'] == 'ёбаный') & (dataset['Частота коллокации'] > 10)].sort_values('MI', ascending=False)[0:50]

,Первое слово,Второе слово,Частота коллокации,Частота первого слова,Частота второго слова,MI
81444,ёбаный,стыд,107,919,310,12.889853
1385011,ёбаный,вт,15,919,94,11.776812
1025090,ёбаный,насос,18,919,121,11.675572
8283,ёбаный,хуйня,139,919,1648,10.856951
5852093,ёбаный,феминистический,11,919,149,10.664773
771514,ёбаный,мать,16,919,3812,6.528178
141306,ёбаный,ты,32,919,29283,4.586734
260836,ёбаный,в,21,919,713630,-0.627989


In [105]:
dataset[(dataset['Второе слово'] == 'хуй') & (dataset['Частота коллокации'] > 10)].sort_values('MI', ascending=False)[0:50]

,Первое слово,Второе слово,Частота коллокации,Частота первого слова,Частота второго слова,MI
112436,пинать,хуй,46,229,3657,10.116344
148117,сосать,хуй,82,602,3657,9.555919
644417,класть,хуй,37,685,3657,8.221479
2028022,стеклянный,хуй,15,335,3657,7.950860
934610,полова,хуй,23,635,3657,7.644935
407453,забить,хуй,33,1040,3657,7.454013
67722,какимтый,хуй,29,1117,3657,7.164554
520820,ясный,хуй,13,607,3657,6.886873
790651,положить,хуй,35,2052,3657,6.558454
2010448,половой,хуй,23,1391,3657,6.513641


In [114]:
dataset[(dataset['Первое слово'] == 'пизда') & (dataset['Частота коллокации'] > 5)].sort_values('MI', ascending=False)[0:50]

,Первое слово,Второе слово,Частота коллокации,Частота первого слова,Частота второго слова,MI
192858,пизда,рукав,10,1243,209,9.603390
1451535,пизда,жж,16,1243,1648,7.302320
928694,пизда,блядь,6,1243,1625,5.907559
1514740,пизда,тупой,6,1243,3350,4.863838
1967320,пизда,дать,6,1243,11853,3.040818
1007493,пизда,получить,6,1243,13763,2.825276
211228,пизда,ты,9,1243,29283,2.320970
827466,пизда,от,25,1243,82775,2.295769
2057799,пизда,под,7,1243,27842,2.031200
319995,пизда,я,12,1243,49199,1.987442
